In [63]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

dir_path = "raw/10ul/"
file_list = [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f))]

class DataHandler:
    def __init__(self):
        pass     

    def import_raw(self, path):
        full_data = pd.read_csv(path)    
        raw_data = pd.DataFrame()
        list_of_features = ['Epoch Time', 'Z1 BlkTemp', 'Z1 HeatSink', 'Z1 Iset', 'Z1 Imea']
        raw_data = pd.concat([raw_data, full_data[list_of_features]], axis=1)    
        return raw_data
    
    def rename_labels(self, raw_data):
        raw_data = raw_data.rename(columns={'Z1 BlkTemp': 'Block Temp',
                                            'Z1 HeatSink': 'Heat Sink Temp',
                                            'Z1 Iset': 'Iset',
                                            'Z1 Imea': 'Imeasure'
                                           })        
        return raw_data
    
    def replace_with_period(self, raw_data):
        # Calculate and use delta_t instead of epoch time
        next_epoch = raw_data['Epoch Time'][1::]
        next_epoch.index -= 1
        
        raw_data['Epoch Time'] = next_epoch - raw_data['Epoch Time']
        raw_data = raw_data.rename(columns={'Epoch Time': 'Period'})
        raw_data = raw_data.dropna()
        
        return raw_data
    
    def add_block_rate(self, raw_data):
        prev_temp = raw_data['Block Temp'][:-1]
        prev_temp.index += 1
        raw_data['Block Rate'] = (raw_data['Block Temp'] - prev_temp) / raw_data['Period']
        raw_data = raw_data.dropna()
        
        return raw_data
    
    def add_new_block_temp(self, raw_data):        
        new_block_temp = raw_data['Block Temp'][1::]
        new_block_temp.index -= 1
        raw_data['New Block Temp'] = new_block_temp
        
        raw_data = raw_data.dropna()
        
        return raw_data
    
    def reorder_labels(self, raw_data):
        raw_data = raw_data[['Period',
                             'Heat Sink Temp',
                             'Block Temp',
                             'Block Rate', 'Iset',
                             'Imeasure',
                             'New Block Temp'
                            ]]
        return raw_data

    def process_data(self, raw_data):      
        raw_data = self.rename_labels(raw_data)
        raw_data = self.replace_with_period(raw_data)
        raw_data = self.add_block_rate(raw_data)
        raw_data = self.add_new_block_temp(raw_data)
        raw_data = self.reorder_labels(raw_data)
        return raw_data

In [64]:
handler = DataHandler()
raw_data = handler.import_raw(file_list[0])
raw_data = handler.rename_labels(raw_data)
raw_data = handler.replace_with_period(raw_data)
raw_data = handler.add_block_rate(raw_data)
raw_data = handler.add_new_block_temp(raw_data)
raw_data = handler.reorder_labels(raw_data)
raw_data

,Period,Heat Sink Temp,Block Temp,Block Rate,Iset,Imeasure,New Block Temp
1,0.2,48.8783,45.2264,7.406498,6.4000,6.2535,46.7303
2,0.2,48.8019,46.7303,7.519498,6.4000,6.1971,48.1923
3,0.2,48.7254,48.1923,7.309998,6.4000,6.2642,49.7221
4,0.2,48.5723,49.7221,7.649007,6.4000,6.1595,51.2365
5,0.2,48.4955,51.2365,7.571998,5.9745,5.9634,52.3360
...,...,...,...,...,...,...,...
1549,0.2,31.9008,58.3200,-2.935499,6.4000,6.1057,57.6463
1550,0.4,31.9540,57.6463,-1.684251,6.4000,6.2051,56.2524
1551,0.2,31.9540,56.2524,-6.969498,6.4000,6.1245,55.5759
1552,0.2,31.9540,55.5759,-3.382499,6.4000,6.0171,54.8421
